<a href="https://colab.research.google.com/github/mkovac03/2020_analysis_of_big_earth_data_with_jupyter/blob/master/GEEexport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
 !pip install earthengine-api
 !pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 4.2 MB/s 
     |████████████████████████████████| 95 kB 3.7 MB/s 
     |████████████████████████████████| 3.0 MB 29.4 MB/s 
     |████████████████████████████████| 98 kB 5.9 MB/s 
     |████████████████████████████████| 187 kB 43.8 MB/s 
     |████████████████████████████████| 1.2 MB 41.6 MB/s 
     |████████████████████████████████| 99 kB 7.0 MB/s 
     |████████████████████████████████| 130 kB 48.0 MB/s 
     |████████████████████████████████| 46 kB 3.6 MB/s 
     |████████████████████████████████| 3.3 MB 26.0 MB/s 
     |████████████████████████████████| 1.3 MB 34.2 MB/s 
     |████████████████████████████████| 8.8 MB 33.3 MB/s 
     |████████████████████████████████| 128 kB 39.5 MB/s 
     |████████████████████████████████| 428 kB 36.2 MB/s 

In [5]:
import ee
import numpy as np
import geemap
import matplotlib.pyplot as plt
import os

In [6]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=j-2xJiK8VXGQFKChNra9Et0Q8_9loHkpD3ksiiHMptE&tc=ByZISeu9_XDBp7Rlsd9HGRwUJiXmNt1rWNStewePmmA&cc=fyAyeDA98CJGmEyX9YV98oPYOQKW23QeyBBaNudT7dw

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhEgWZYHWWarv6csv_U20BG5E9WVIwF74mR9mzH4jC-Ar0zDtmFJBg

Successfully saved authorization token.


In [7]:
Map = geemap.Map()

countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')

aoi = ee.Feature(countries.filter(ee.Filter.inList('country_na',
 ['South Africa'])).union().first()).geometry()

Map.addLayer(aoi)
# Zoom to regions of interest
Map.centerObject(aoi, 6)

dataset = ee.ImageCollection('MODIS/061/MOD13Q1') \
                  .filter(ee.Filter.date('2000-01-01', '2000-12-31')) \ # change this range to the desired dates
                  .map(lambda image: image.clip(aoi)) \
                  .select('NDVI')

ndviVis = {
  'min': 0.0,
  'max': 8000.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
}
Map.addLayer(dataset.first(), ndviVis, 'NDVI')

grids = dataset.geometry().coveringGrid('EPSG:4326', 100000)
Map.addLayer(grids, {}, 'grid')
Map

Map(center=[-28.975081121649072, 25.20179946159211], controls=(WidgetControl(options=['position', 'transparent…

In [8]:
n_img = dataset.size().getInfo()
print(f'number of images: {n_img}')

n_feat = grids.size().getInfo()
print(f'number of grids: {n_feat}')

number of images: 20
number of grids: 185


In [9]:
imgList = dataset.toList(n_img)
featList = grids.toList(n_feat)

In [10]:
import time
for i in range(n_img):
    for n in range(n_feat):
        grid_n = ee.Feature(featList.get(n)).buffer(distance = 500).bounds()
        task = ee.batch.Export.image.toDrive(
            image = ee.Image(imgList.get(i)).clip(grid_n),
            description= f"{imgList.get(i).getInfo()['id'].replace('/', '_')}_sa_{n}",
            region= grid_n.geometry(),
            scale= 250,
            maxPixels= 10e12,
            # change this to the desired folder name on Google Drive
            folder= 'MODIS_NDVI_SA',
            crs='EPSG:4326',
            fileFormat='GeoTIFF')
        task.start()
        print(f"Exporting {imgList.get(i).getInfo()['id'].replace('/', '_')}_sa_{n} to Drive...")
        while task.active():
            time.sleep(30)
        if task.status()['state'] != 'COMPLETED':
            print('Error with export.')
        else:
            print('Export completed.')

Exporting MODIS_061_MOD13Q1_2000_02_18_sa_0 to Drive...
Export completed.
Exporting MODIS_061_MOD13Q1_2000_02_18_sa_1 to Drive...


KeyboardInterrupt: ignored